# Test and results.

In [ ]:
from HMM.Markovify import *
from HMM.Utils.CrossValidation import cross_validation_score

from nltk.corpus import brown

import pandas as pd
import numpy as np
import random
from math import ceil
import importlib
import json

from HMM.Utils.ProgressBar import log_progress

# Accuracy for news text.

In [ ]:
corpus = brown.tagged_sents(categories='news', tagset='universal')

model = Markovify(smoothing='max')
brown_news_score = cross_validation_score(model, sentences=corpus, cv=10, verbose=False)
print(json.dumps(brown_news_score, indent=1))
print('Best accuracy: {}.'.format(np.max([model['accuracy'] for model in brown_news_score])))

In [ ]:
sample_idx = random.sample(range(0, len(corpus)), ceil(len(corpus) * 0.2))
sample_corpus = list(corpus[i] for i in sample_idx)

nfolds = [3, 5, 10]
scores = []
for n in log_progress(nfolds):
    model = Markovify(smoothing='max')
    model_scores = cross_validation_score(model, sentences=sample_corpus, cv=n, verbose=False)
    scores.append(np.max([model['accuracy'] for model in model_scores]))
    
print(scores)

# Comparison of different smoothings.

In [ ]:
corpus = brown.tagged_sents(categories='news', tagset='universal')

model_laplace = Markovify(smoothing='laplace', alpha=1)
brown_news_accuracy_laplace = cross_validation_score(model_laplace, sentences=corpus, verbose=False)
print('Best accuracy for Laplace smoothing: {}.'.format(np.max([model['accuracy'] for model in brown_news_accuracy_laplace])))

model_max = Markovify(smoothing='max')
brown_news_accuracy_max = cross_validation_score(model_max, sentences=corpus, verbose=False)
print('Best accuracy for most probable tag: {}.'.format(np.max([model['accuracy'] for model in brown_news_accuracy_max])))

# Accuracy for news text training the model using an adventures book.

In [ ]:
corpus_advs = brown.tagged_sents(categories='adventure', tagset='universal')
corpus_news = brown.tagged_sents(categories='news', tagset='universal')

model = Markovify(smoothing='max')
model = model.fit(corpus_news)

tagged = model.predict(corpus_advs)

In [ ]:
advs_records = [item for sublist in corpus_advs for item in sublist]
predicted_tags = [item for sublist in tagged for item in sublist]

results = pd.DataFrame.from_records(advs_records)
results.columns = ["Word", "Tag"]
results['Predicted'] = predicted_tags
results.head(10)

In [ ]:
from HMM.Utils.Scoring import *

cm = confusion_matrix(corpus_advs, tagged)

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sn
plt.rcParams['figure.figsize'] = [10, 5]

sn.heatmap(cm)

In [ ]:
print(accuracy(corpus_advs, tagged))
print(recall(corpus_advs, tagged))
print(precision(corpus_advs, tagged))

# Accuracy of the model based on the size of the training corpus.

In [ ]:
corpus = brown.tagged_sents(categories='adventure', tagset='universal')

corpus_len = len(corpus)
train_size = [  1 * corpus_len / 100, 
                5 * corpus_len / 100, 
               10 * corpus_len / 100, 
               25 * corpus_len / 100, 
               50 * corpus_len / 100, 
               80 * corpus_len / 100, 
              100 * corpus_len / 100]

accuracies = []
for length in log_progress(train_size):
    partial_corpus = list(corpus[i] for i in range(0, ceil(length)))

    m = Markovify(smoothing='max')
    fold_scores = cross_validation_score(m, sentences=partial_corpus)
    accuracies.append(np.max([fold['accuracy'] for fold in fold_scores]))
    
print(accuracies)



In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt

plt.plot(train_size, accuracies)

In [ ]:
corpus = brown.tagged_sents(categories='adventure', tagset='universal')

m = Markovify(smoothing='max')
acc, m = cross_validation_score(m, sentences=corpus, return_model=True)

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt

tags = m.hmm.q
pos = np.arange(len(tags))
counts = m.hmm.tag_count

plt.barh(pos, counts, align='center', alpha=0.5)
plt.yticks(pos, tags)
plt.xlabel('Recuento del tag')

In [ ]:
counts